In [ ]:
# 필요한 라이브러리 설치 lightfm-1.17
!pip install lightfm pandas

In [1]:
# 라이브러리 임포트
from lightfm import LightFM
from lightfm.data import Dataset
import pandas as pd
import numpy as np
from lightfm.evaluation import precision_at_k

C:\Users\DaeunJang\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [ ]:
# 각 라이브러리의 버전 출력
print("LightFM version:", lightfm.__version__)
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)

In [2]:
# 데이터 로드
df = pd.read_csv('final_animedataset/final_animedataset.csv')

In [3]:
df

,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre
0,karthiga,21,9,2255153,Female,One Piece,TV,Manga,8.54,423868,91.0,35,"Action, Adventure, Comedy, Super Power, Drama,..."
1,karthiga,59,7,2255153,Female,Chobits,TV,Manga,7.53,175388,1546.0,188,"Sci-Fi, Comedy, Drama, Romance, Ecchi, Seinen"
2,karthiga,74,7,2255153,Female,Gakuen Alice,TV,Manga,7.77,33244,941.0,1291,"Comedy, School, Shoujo, Super Power"
3,karthiga,120,7,2255153,Female,Fruits Basket,TV,Manga,7.77,167968,939.0,222,"Slice of Life, Comedy, Drama, Romance, Fantasy..."
4,karthiga,178,7,2255153,Female,Ultra Maniac,TV,Manga,7.26,9663,2594.0,2490,"Magic, Comedy, Romance, School, Shoujo"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35305690,Destoria-LeVaun,3250,10,266038,Female,Daishikkin Helena,OVA,Original,5.95,1245,NaN,5919,Hentai
35305691,MariiTrindade_,4340,0,537855,Female,Tower of Etruria,OVA,Manga,5.85,301,NaN,8194,"Adventure, Comedy, Harem, Hentai"
35305692,mariapeterson,4515,6,1433359,Female,Seton Doubutsuki: Risu no Banner,TV,Unknown,6.69,316,4813.0,8617,"Kids, Drama"
35305693,Nemo584,4730,10,443290,Male,Inyoku Tokkyuu Zetsurinou,OVA,Original,5.95,822,NaN,6069,Hentai


In [4]:
# 유저와 아이템 ID를 LightFM 데이터셋에 맞게 매핑
dataset = Dataset()
dataset.fit(users=df['user_id'].unique(), items=df['title'].unique())

In [ ]:
# 상호작용 매트릭스 구축
(interactions, _) = dataset.build_interactions([(x['user_id'], x['title'])
                                                 for _, x in df.iterrows()])

In [ ]:
# LightFM 모델 생성 및 트레이닝
model = LightFM(loss='warp')
model.fit(interactions, epochs=30, num_threads=2)



In [ ]:
# 새로운 유저 정보 입력받기
new_user_name = input("Enter your name: ")
new_user_gender = input("Enter your gender: ")
new_user_pref_source = input("Enter your preferred source (tv/manga/lightnovel): ")

# 유저 ID 생성 (예시)
new_user_id = max(df['user_id']) + 1

In [ ]:
# 선호 타입에 맞는 아이템 필터링
filtered_items = df[df['source'] == new_user_pref_source]['title'].unique()
filtered_item_ids = dataset.mapping()[2][filtered_items]

# 추천 함수 정의
def get_recommendations(model, user_id, item_ids, num_items=3):
    scores = model.predict(user_id, item_ids)
    top_items_indices = np.argsort(-scores)[:num_items]
    return item_ids[top_items_indices]

# 추천 아이템 얻기
recommended_items = get_recommendations(model, new_user_id, filtered_item_ids, num_items=3)
recommended_titles = [df[df['title'] == title]['title'].iloc[0] for title in recommended_items]
print("Recommended items for new user {}: {}".format(new_user_id, recommended_titles))

# 성능 평가 (정확도)
train_precision = precision_at_k(model, interactions, k=3).mean()
print("Train precision at k=3: {:.2f}".format(train_precision))